<a href="https://colab.research.google.com/github/PharahMain/Flight-Delay-Predictor/blob/master/flightstats_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [5]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 58kB/s 
     |████████████████████████████████| 204kB 42.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=95cc22c88d22c4bbf015d8a8c7e997c7f1d8f9485502b1216f6390f8f9716942
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [2]:
# Point Colaboratory to Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col, create_map, lit
from pyspark.sql.functions import month, dayofweek, weekofyear, hour, minute
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType, TimestampType

from datetime import datetime, date, timedelta
from itertools import chain
import random
import time

In [0]:
#ARR_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/{date.today()}.json"
# for testing
ARR_JSON_PATH = "/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/2020-01-17.json"

#DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/{date.today()}.json"
DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/2020-01-17.json"

#WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/{date.today()}.json"
WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/2020-01-17.json"

APP_NAME = 'Flight Delay Predictor'
SPARK_URL = 'local[*]'
RANDOM_SEED = 263646
TRAINING_DATA_RATIO = 0.7

In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [12]:
df_arr = spark.read.json(ARR_JSON_PATH)
df_arr.show(5)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|    airportResources|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          codeshares|              delays|departureAirportFsCode|       departureDate|divertedAirportFsCode|     flightDurations|   flightEquipment|  flightId|flightNumber|    operationalTimes|            schedule|status|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|      [, TBIT,, C9,]|                 LAX|[2020-01-17T13:10...|           QR|     [[3515, UL, L]]| 

The data in a little more detail.

In [13]:
df_arr.head(5)

[Row(airportResources=Row(arrivalGate=None, arrivalTerminal='TBIT', baggage=None, departureGate='C9', departureTerminal=None), arrivalAirportFsCode='LAX', arrivalDate=Row(dateLocal='2020-01-17T13:10:00.000', dateUtc='2020-01-17T21:10:00.000Z'), carrierFsCode='QR', codeshares=[Row(flightNumber='3515', fsCode='UL', relationship='L')], delays=Row(arrivalGateDelayMinutes=None, arrivalRunwayDelayMinutes=13, departureGateDelayMinutes=52, departureRunwayDelayMinutes=60), departureAirportFsCode='DOH', departureDate=Row(dateLocal='2020-01-17T07:45:00.000', dateUtc='2020-01-17T04:45:00.000Z'), divertedAirportFsCode=None, flightDurations=Row(blockMinutes=None, scheduledAirMinutes=1021, scheduledBlockMinutes=985, scheduledTaxiInMinutes=None, scheduledTaxiOutMinutes=10, taxiOutMinutes=18), flightEquipment=Row(actualEquipmentIataCode='77L', scheduledEquipmentIataCode='77L', tailNumber='A7-BBA'), flightId=1028199581, flightNumber='739', operationalTimes=Row(actualGateArrival=None, actualGateDeparture

Let's see how many flights arrived at LAX in the past 24 hours.

In [14]:
df_arr.count()

337

Let's first drop all the unnecessary columns.

In [0]:
df_arr = df_arr.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')
df_arr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|
+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+
|                 LAX|[2020-01-17T13:10...|           QR|      [, 13, 52, 60]|                   DOH|[, 1021, 985,, 10...|[77L, 77L, A7-BBA]|
|                 LAX|[2020-01-17T13:15...|           QF|[205, 247, 226, 240]|                   SYD|[, 763, 830, 57, ...|[388, 388, VH-OQJ]|
|                 LAX|[2020-01-17T13:50...|           SU|          [14, 23,,]|                   SVO|[, 745, 770, 15, ...|[332, 332, VQ-BBE]|
|                 LAX|[2020-01-17T14:00...|           NZ|       [, 7, 17, 25]|                   AKL|[, 681, 725, 34, ...|[77W, 77W, ZK-OKS]|
|     

In [0]:
df_arr.printSchema()

root
 |-- arrivalAirportFsCode: string (nullable = true)
 |-- arrivalDate: struct (nullable = true)
 |    |-- dateLocal: string (nullable = true)
 |    |-- dateUtc: string (nullable = true)
 |-- carrierFsCode: string (nullable = true)
 |-- delays: struct (nullable = true)
 |    |-- arrivalGateDelayMinutes: long (nullable = true)
 |    |-- arrivalRunwayDelayMinutes: long (nullable = true)
 |    |-- departureGateDelayMinutes: long (nullable = true)
 |    |-- departureRunwayDelayMinutes: long (nullable = true)
 |-- departureAirportFsCode: string (nullable = true)
 |-- flightDurations: struct (nullable = true)
 |    |-- blockMinutes: long (nullable = true)
 |    |-- scheduledAirMinutes: long (nullable = true)
 |    |-- scheduledBlockMinutes: long (nullable = true)
 |    |-- scheduledTaxiInMinutes: long (nullable = true)
 |    |-- scheduledTaxiOutMinutes: long (nullable = true)
 |    |-- taxiOutMinutes: long (nullable = true)
 |-- flightEquipment: struct (nullable = true)
 |    |-- actualEq

Let's explore the dataframe a little bit here.

In [0]:
df_arr.groupBy('carrierFsCode').count().orderBy('count', ascending=False).show()

+-------------+-----+
|carrierFsCode|count|
+-------------+-----+
|           DL|   48|
|           AA|   45|
|           OO|   44|
|           WN|   42|
|           CP|   30|
|           UA|   25|
|           AS|   24|
|           NK|    6|
|           B6|    6|
|           QX|    5|
|           AC|    4|
|           WS|    3|
|           Y4|    3|
|           QF|    3|
|           BA|    3|
|           NZ|    2|
|           CZ|    2|
|           1I|    2|
|           K4|    2|
|          BAI|    2|
+-------------+-----+
only showing top 20 rows



Let's see what the busiest time of the day is for LAX next. But before we do that, I'm going to separate out the arrival time from 'arrivalDate'.

In [0]:
# first need to extract the date info from the arrivalDate struct
df_arr = df_arr.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
#df_arr.select('localDate').show(5)

df_arr = df_arr.withColumn('localTimeStamp', df_arr['localDate'].cast(TimestampType()))
df_arr.select('localTimeStamp').show(5)

+-------------------+
|     localTimeStamp|
+-------------------+
|2020-01-17 13:10:00|
|2020-01-17 13:15:00|
|2020-01-17 13:50:00|
|2020-01-17 14:00:00|
|2020-01-17 14:40:00|
+-------------------+
only showing top 5 rows



Give each time increment its own column.

In [0]:
df_arr.withColumn('date', col('localTimeStamp').cast('date')).withColumn('hour', hour(col('localTimeStamp'))).withColumn('minute', minute(col('localTimeStamp'))).show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+----------+----+------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|           localDate|     localTimeStamp|      date|hour|minute|
+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+----------+----+------+
|                 LAX|[2020-01-17T13:10...|           QR|      [, 13, 52, 60]|                   DOH|[, 1021, 985,, 10...|[77L, 77L, A7-BBA]|2020-01-17T13:10:...|2020-01-17 13:10:00|2020-01-17|  13|    10|
|                 LAX|[2020-01-17T13:15...|           QF|[205, 247, 226, 240]|                   SYD|[, 763, 830, 57, ...|[388, 388, VH-OQJ]|2020-01-17T13:15:...|2020-01-17 13:

Since travel tends to change with the time of year, let's also add on weekOfYear and dayOfWeek as well.

In [0]:
df_arr.withColumn('weekOfYear', weekofyear(col('localTimeStamp'))).withColumn('dayOfWeek', dayofweek(col('localTimeStamp'))).show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+----------+---------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|           localDate|     localTimeStamp|weekOfYear|dayOfWeek|
+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+----------+---------+
|                 LAX|[2020-01-17T13:10...|           QR|      [, 13, 52, 60]|                   DOH|[, 1021, 985,, 10...|[77L, 77L, A7-BBA]|2020-01-17T13:10:...|2020-01-17 13:10:00|         3|        6|
|                 LAX|[2020-01-17T13:15...|           QF|[205, 247, 226, 240]|                   SYD|[, 763, 830, 57, ...|[388, 388, VH-OQJ]|2020-01-17T13:15:...|2020-01-17 13:15:00|  

Delays column, the column that contains our target data, is an array of four values. They are minutes of delay in different stages of the flight but, since travelers don't really care where the delays occur, but rather how long the delays are, let's sum up the numbers to get one total delayed minutes.

In [0]:
# a df with just the delay data
delay_df = df_arr.select('delays.*').na.fill(0)
delay_df.show(5)

+-----------------------+-------------------------+-------------------------+---------------------------+
|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|
+-----------------------+-------------------------+-------------------------+---------------------------+
|                      0|                       13|                       52|                         60|
|                    205|                      247|                      226|                        240|
|                     14|                       23|                        0|                          0|
|                      0|                        7|                       17|                         25|
|                      0|                        0|                        0|                          0|
+-----------------------+-------------------------+-------------------------+---------------------------+
only showing top 5 rows



In [0]:
# create a new column in main arrival df with total delay minutes
delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

delay_df.show(5)


+-----------------------+-------------------------+-------------------------+---------------------------+----------+----+
|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|d_id|
+-----------------------+-------------------------+-------------------------+---------------------------+----------+----+
|                      0|                       13|                       52|                         60|       125|   0|
|                    205|                      247|                      226|                        240|       918|   1|
|                     14|                       23|                        0|                          0|        37|   2|
|                      0|                        7|                       17|                         25|        49|   3|
|                      0|                        0|                        0|                          0|         0|   4|
+-----------------------

Now let's merge this table with the main table.

In [0]:
# In order to join two dataframes, we need to create an index column to join the two 
# dataframes on
df_arr = df_arr.select('*').withColumn('id', monotonically_increasing_id())

df_arr = df_arr.join(delay_df, col('id') == col('d_id'), how='leftouter').drop('d_id')
df_arr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+---+-----------------------+-------------------------+-------------------------+---------------------------+----------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|           localDate|     localTimeStamp| id|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|
+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+---+-----------------------+-------------------------+-------------------------+---------------------------+----------+
|                 LAX|[2020-01-17T13:10...|           QR|      [, 13, 52, 60]|                

Now let's do the same for flightDurations and flightEquipment.

In [0]:
eq_df = df_arr.select('flightEquipment.*').drop('tailNumber')
eq_df.show(5)

+-----------------------+--------------------------+
|actualEquipmentIataCode|scheduledEquipmentIataCode|
+-----------------------+--------------------------+
|                    77L|                       77L|
|                    388|                       388|
|                    332|                       332|
|                    77W|                       77W|
|                    788|                       788|
+-----------------------+--------------------------+
only showing top 5 rows



When it comes to equipment, when it is changed it might cause a delay so I'm going to drop the tailNumber and create another column that shows whether there was a change in equipment.

In [0]:
eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0)
    .otherwise(1))

eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())
eq_df.show(5)

+-----------------------+--------------------------+---------+----+
|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|e_id|
+-----------------------+--------------------------+---------+----+
|                    77L|                       77L|        0|   0|
|                    388|                       388|        0|   1|
|                    332|                       332|        0|   2|
|                    77W|                       77W|        0|   3|
|                    788|                       788|        0|   4|
+-----------------------+--------------------------+---------+----+
only showing top 5 rows



And lastly, let's just some up the flight durations.

In [0]:
fd_df = df_arr.select('flightDurations.*').na.fill(0)
fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())
fd_df.show(5)

+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+----+
|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|f_id|
+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+----+
|           0|               1021|                  985|                     0|                     10|            18|         2034|   0|
|           0|                763|                  830|                    57|                     10|            24|         1684|   1|
|           0|                745|                  770|                    15|                     10|            23|         1563|   2|
|           0|                681|                  725|                    34|                     10|            18|         1468|   3|
|           0|                683|

Put all the tables together.

In [0]:
df_arr = df_arr.join(eq_df, col('id') == col('e_id'), how='leftouter').drop('e_id').join(fd_df, col('id') == col('f_id'), how='leftouter').drop('f_id')
df_arr.orderBy('id').show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+-------------------+---+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|           localDate|     localTimeStamp| id|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipme

In [0]:
df_arr.count()

337

We now need to repeat the same process for the departing flights data. It might be better if I wrote a function that does this given a dataframe.

In [0]:
def format_flight_data(df):
    # lets' first index the df since we know we will be needing it later
    df = df.select('*').withColumn('id', monotonically_increasing_id())

    ##### timestamp #####
    # first need to extract the date info from the arrivalDate struct
    df = df.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
    df = df.withColumn('localTimeStamp', df['localDate'].cast(TimestampType()))
    df = df.withColumn('date', col('localTimeStamp').cast('date'))
    df = df.withColumn('hour', hour(col('localTimeStamp')))
    df = df.withColumn('minute', minute(col('localTimeStamp')))
    df = df.withColumn('weekOfYear', weekofyear(col('localTimeStamp')))
    df = df.withColumn('dayOfWeek', dayofweek(col('localTimeStamp')))

    ##### delays #####
    # fill all null values with 0
    delay_df = df.select('delays.*').na.fill(0)
    # create a new column in main arrival df with total delay minutes
    delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
    delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

    ##### equipment #####
    eq_df = df.select('flightEquipment.*').drop('tailNumber')
    eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0).otherwise(1))
    eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())

    ##### duration #####
    fd_df = df.select('flightDurations.*').na.fill(0)
    fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
    fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())

    # join all dataframes
    formatted_df = df.join(delay_df, col('id')==col('d_id'), how='leftouter').drop('d_id')
    formatted_df = formatted_df.join(eq_df, col('id')==col('e_id'), how='leftouter').drop('e_id')
    formatted_df = formatted_df.join(fd_df, col('id')==col('f_id'), how='leftouter').drop('f_id')

    # drop extraneous columns
    formatted_df = formatted_df.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')

    return formatted_df

    

In [17]:
df_arr_fmtd = format_flight_data(df_arr)
df_arr_fmtd.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|s

Nice. Now let's format the departing dataframe.

In [18]:
df_dep = spark.read.json(DEP_JSON_PATH)
df_dep_fmtd = format_flight_data(df_dep)
df_dep_fmtd.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDura

Now that we have a formatted dataframe for both arriving and departing flights, we need to join the weather conditions for each flight. The data from the DarkSky API contains within it a timestamp but it is in UNIX time. So now we just need to convert it so that it matches up with the formatted dataframe and do an inner join.

In [19]:
df_weath = spark.read.json(WEATH_JSON_PATH)
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|
|               53.2|      0.88|   49.85|    0.88|               rain|303.5|         0.0262|            

In [20]:
df_weath = df_weath.withColumn('f_time', df_weath['time'].cast(TimestampType()))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|2020-01-17 08:00:00|
|               53.2|   

Hmmm. Since the weather data is taken hourly, we now just need to match upto the hour.

In [21]:
# separate date and hour
df_weath = df_weath.withColumn('w_date', col('f_time').cast('date'))
df_weath = df_weath.withColumn('w_hour', hour(col('f_time')))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|    w_date|w_hour|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85| 

Now all that's left to do is inner join the weather data on both the date and hour columns.

In [22]:
df_arr_wthr = df_arr_fmtd.join(df_weath, (col('date')==col('w_date'))&(col('hour')==col('w_hour')), how='inner')
df_arr_wthr = df_arr_wthr.drop('time', 'w_date', 'w_hour')
df_arr_wthr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrival

In [23]:
df_dep_wthr = df_dep_fmtd.join(df_weath, (col('date')==col('w_date')) & (col('hour')==col('w_hour')), how='inner')
df_dep_wthr = df_dep_wthr.drop('time', 'w_date', 'w_hour')
df_dep_wthr.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes

Next, I think I will select only the feature and target columns.

In [24]:
df_arr_final = df_arr_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment', 
                               'localDate', 'localTimeStamp', 'icon', 'f_time')
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|

In [25]:
# rinse and repeat for the departing flights
df_dep_final = df_dep_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment',
                                'localDate', 'localTimeStamp', 'icon', 'f_time')
df_dep_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|hu

While we have the columns narrowed down, we still have a long way to go to be ready to build a model. Since we will be using linear regression as one of the methods, we have to encode all the categorical variables.

In [26]:
# let's first create a dictionary of all the carriers present in the dataframe
# get unique values from column
unique_airlines = df_arr_final.select('carrierFsCode').distinct()

# make a list of all airlines
airline_list = [row.carrierFsCode for row in unique_airlines.collect()]

# create a random sampling of integer ids
random_codes = random.sample(range(1000), len(airline_list))

# assign each random int id to an airline
airline_dict = dict(zip(airline_list, random_codes))
print(airline_dict)


{'XOJ': 33, 'CI': 175, 'BAI': 324, 'UA': 370, 'NK': 814, 'AA': 951, 'AV': 817, 'QF': 39, 'CZ': 586, 'B6': 530, 'CX': 691, 'NH': 715, 'CV': 284, 'SU': 44, 'LX': 622, 'DL': 2, 'CA': 1, 'MW': 491, 'BR': 665, 'OO': 637, 'F9': 726, 'VB': 23, 'LH': 876, 'CM': 152, 'Y4': 335, 'VS': 42, 'HA': 694, 'BA': 840, 'LP': 188, 'EI': 465, 'WS': 97, 'DI*': 209, 'LO': 972, 'MU': 696, '5X': 581, 'G4': 341, 'MF': 227, 'NZ': 96, 'CP': 139, 'QR': 824, 'OZ': 54, 'KE': 879, '1I': 785, 'AS': 274, 'AC': 572, 'QX': 473, 'K4': 931, 'TK': 747, 'PR': 280, 'TA': 907, 'SQ': 801, 'AF': 926, 'WN': 203, 'L1E': 702}


In [27]:
total_flts = df_arr_final.count()
unique_from_stn = df_arr_final.select('departureAirportFsCode').distinct()
from_stn_total = unique_from_stn.count()
total_alines = len(airline_list)
arr_airpt = df_arr_final.first().arrivalAirportFsCode

print(f'There are {total_flts} flights from {from_stn_total} different stations operated by {total_alines} different airlines arriving at {arr_airpt} today.')

There are 337 flights from 123 different stations operated by 54 different airlines arriving at LAX today.


Now that we have a dictionary, let's apply it to our dataframe.

In [28]:
map_exp = create_map([lit(x) for x in chain(*airline_dict.items())])

df_arr_final = df_arr_final.withColumn('a_code', map_exp.getItem(col('carrierFsCode')))
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutM

Looking through the dataframe, we have a couple more columns that needs to go under the same process to encode.

In [29]:
# originating stations
from_stn_list = [row.departureAirportFsCode for row in unique_from_stn.collect()]
random_codes_2 = random.sample(range(1001, 2000), len(from_stn_list))
from_stn_dict = dict(zip(from_stn_list, random_codes_2))
print(from_stn_dict)

{'MSY': 1549, 'GEG': 1158, 'EUG': 1233, 'OAK': 1969, 'BNE': 1988, 'MSN': 1620, 'DCA': 1371, 'QRO': 1602, 'RDM': 1765, 'PEK': 1996, 'CMH': 1447, 'CDG': 1814, 'SIN': 1907, 'IAH': 1077, 'MZT': 1372, 'SYD': 1446, 'HNL': 1432, 'SJC': 1001, 'BUF': 1473, 'PVG': 1673, 'AUS': 1197, 'MUC': 1985, 'MEX': 1433, 'ZIH': 1535, 'RNO': 1016, 'JAC': 1048, 'IPL': 1470, 'TUL': 1228, 'BOS': 1414, 'EWR': 1383, 'DOH': 1205, 'LAS': 1776, 'YEG': 1591, 'LIR': 1080, 'BCN': 1756, 'XNA': 1617, 'YYZ': 1887, 'DEN': 1942, 'BJX': 1904, 'IAD': 1603, 'BOI': 1650, 'SBA': 1588, 'SJD': 1772, 'SEA': 1373, 'MNL': 1570, 'ICN': 1151, 'MCI': 1938, 'WAW': 1670, 'CLT': 1567, 'BNA': 1849, 'PAE': 1261, 'YVR': 1469, 'CUN': 1921, 'ABQ': 1225, 'SVO': 1779, 'SDF': 1890, 'MEL': 1923, 'SCK': 1063, 'DAL': 1721, 'MRY': 1292, 'PDX': 1154, 'PVR': 1978, 'LGW': 1801, 'MIA': 1764, 'MFR': 1192, 'LHR': 1892, 'DGO': 1838, 'PRC': 1084, 'SAL': 1880, 'OKC': 1940, 'YYC': 1496, 'SMF': 1889, 'PTY': 1013, 'PHX': 1272, 'STL': 1323, 'DFW': 1632, 'MMH': 1994

In [40]:
# aircrafts(ac)
ac_actual = df_arr_final.select('actualEquipmentIataCode').na.fill('None').distinct()
ac_sched = df_arr_final.select('scheduledEquipmentIataCode').na.fill('None').distinct()

ac_actual_list = [row.actualEquipmentIataCode for row in ac_actual.collect()]
ac_sched_list = [row.scheduledEquipmentIataCode for row in ac_sched.collect()]

ac_combo_list = list(set(ac_actual_list + ac_sched_list))

random_codes_3 = random.sample(range(2001, 3000), len(ac_combo_list))

ac_dict = dict(zip(ac_combo_list, random_codes_3))
print(ac_dict)

print(f'A total of {len(ac_combo_list)} different types of aircraft operating today.')

{'75W': 2110, '32S': 2472, '77L': 2313, 'E75': 2309, '773': 2428, '32B': 2016, '74F': 2026, '319': 2666, '747': 2397, '788': 2204, '74Y': 2762, 'CRJ': 2240, '772': 2903, '346': 2211, '77F': 2794, '330': 2327, 'CNA': 2590, 'CR7': 2854, '74N': 2156, '7S8': 2308, '32N': 2886, '738': 2710, '752': 2368, 'CNJ': 2533, '388': 2352, '320': 2457, '757': 2285, '77X': 2917, 'PL2': 2600, '321': 2611, '77W': 2527, '753': 2415, 'CN2': 2098, '31B': 2828, '76V': 2459, '744': 2678, '741': 2116, '739': 2816, '32Q': 2597, '781': 2284, '789': 2785, '73W': 2001, '76W': 2696, '359': 2817, '75D': 2676, '73R': 2323, '73H': 2637, '73J': 2471, '332': 2962, '73G': 2974, '75T': 2387, '763': 2629, 'E7W': 2665, '32A': 2005, 'CJL': 2229, 'CR2': 2920, 'None': 2484}
A total of 57 different types of aircraft operating today.


In [33]:
# precipitation
df_arr_final.select('precipType').distinct().show()


+----------+
|precipType|
+----------+
|      null|
|      rain|
+----------+



It looks like we can just drop this column altogether.

In [0]:
df_arr_final = df_arr_final.drop('precipType', 'summary')

In [38]:
df_arr_final.show(1)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|appa

In [0]:
map_exp_2 = create_map([lit(x) for x in chain(*from_stn_dict.items())])
map_exp_3 = create_map([lit(x) for x in chain(*ac_dict.items())])

In [0]:
df_arr_final = df_arr_final.withColumn('fm_code', map_exp_2.getItem(col('departureAirportFsCode')))
df_arr_final = df_arr_final.withColumn('aac_code', map_exp_3.getItem(col('actualEquipmentIataCode')))
df_arr_final = df_arr_final.withColumn('sac_code', map_exp_3.getItem(col('scheduledEquipmentIataCode')))

In [45]:
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOut

Once the columns containing non-numerical values are dropped, we should finally have a dataframe we can model and predict with.

In [46]:
df_arr_num = df_arr_final.drop('arrivalAirportFsCode', 'carrierFsCode', 'departureAirportFsCode', 'date', 'id',
                               'actualEquipmentIataCode', 'scheduledEquipmentIataCode')
df_arr_num.show(5)

+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|humidity|ozone|precipIntensity|precipProbability|pressure|temperature|uvIndex|visibility|windBearing|windGust|windSpeed|a_code|fm_code|aac_code|sac_code|
+----+------+----------+--------